In [ ]:
# default_exp eumetsat

# EUMETSAT API Wrapper Development

In [ ]:
#export
import numpy as np
import pandas as pd
import dataset

import FEAutils as hlp
from typing import Union, List
import xmltodict
import dotenv
import datetime
import zipfile
import copy
import os
from io import BytesIO
import re
import glob
import logging
import math
import shutil
import subprocess
from pathlib import Path


from requests.auth import HTTPBasicAuth
import requests

from nbdev.showdoc import *
from fastcore.test import test

from satip import utils
from satip.gcp_helpers import get_eumetsat_filenames, upload_blob

from ipypb import track
from IPython.display import JSON

<br>

### User Input

In [ ]:
data_dir = '../data/raw'
compressed_dir = '../data/compressed'
debug_fp = '../logs/EUMETSAT_download.txt'
env_vars_fp = '../.env'
metadata_db_fp = '../data/EUMETSAT_metadata.db'

download_data = True

<br>

### Authorising API Access

First we'll load the the environment variables

In [ ]:
dotenv.load_dotenv(env_vars_fp)

user_key = os.environ.get('USER_KEY')
user_secret = os.environ.get('USER_SECRET')
slack_id = os.environ.get('SLACK_ID')
slack_webhook_url = os.environ.get('SLACK_WEBHOOK_URL')

<br>

And test they were loaded successfully

In [ ]:
def check_env_vars_have_loaded(env_vars):
    for name, value in env_vars.items():
        assert value is not None, f'{name}` should not be None'
    
    return

env_vars = {
    'user_key': user_key,
    'user_secret': user_secret,
    'slack_id': slack_id,
    'slack_webhook_url': slack_webhook_url
}

check_env_vars_have_loaded(env_vars)

<br>

We'll then use them to request an access token for the API

In [ ]:
#export
def request_access_token(user_key, user_secret):
    """
    Requests an access token from the EUMETSAT data API
    
    Parameters:
        user_key: EUMETSAT API key
        user_secret: EUMETSAT API secret
        
    Returns:
        access_token: API access token
        
    """
    
    token_url = 'https://api.eumetsat.int/token'

    data = {
      'grant_type': 'client_credentials'
    }

    r = requests.post(token_url, data=data, auth=(user_key, user_secret))
    access_token = r.json()['access_token']

    return access_token

<br>

We'll then use them to request an access token for the API

In [ ]:
access_token = request_access_token(user_key, user_secret)

<br>

### Querying Available Data

Before we can download any data we have to know where it's stored. To learn this we can query their search-products API, which returns a JSON containing a list of file metadata.

In [ ]:
#export
format_dt_str = lambda dt: pd.to_datetime(dt).strftime('%Y-%m-%dT%H:%M:%SZ')

def query_data_products(
    start_date:str='2020-01-01', 
    end_date:str='2020-01-02', 
    start_index:int=0, 
    num_features:int=10_000,
    product_id:str='EO:EUM:DAT:MSG:MSG15-RSS'
) -> requests.models.Response:
    """
    Queries the EUMETSAT data API for the specified data
    product and date-range. The dates will accept any
    format that can be interpreted by `pd.to_datetime`.
    A maximum of 10,000 entries are returned by the API
    so the indexes of the returned entries can be specified.
    
    Parameters:
        start_date: Start of the query period
        end_date: End of the query period
        start_index: Starting index of returned entries
        num_features: Number of returned entries
        product_id: ID of the EUMETSAT product requested
        
    Returns:
        r: Response from the request
        
    """
    
    search_url = 'https://api.eumetsat.int/data/search-products/os'

    params = {
        'format': 'json',
        'pi': product_id,
        'si': start_index, 
        'c': num_features,
        'sort': 'start,time,0',
        'dtstart': format_dt_str(start_date),
        'dtend': format_dt_str(end_date)
    }

    r = requests.get(search_url, params=params)
    
    assert r.ok, f'Request was unsuccesful - Error code: {r.status_code}'
    
    return r

<br>

We'll quickly make a test request to this end-point

In [ ]:
start_date = '2019-10-01'
end_date = '2019-10-07'

r = query_data_products(start_date, end_date)

r_json = r.json()
JSON(r_json)

<br>

However the search-api is capped (at 10,000) for the number of files it will return metadata for, so we'll create a while loop that waits until all the relevant data has been returned. We'll then extract just the list of features from the returned JSONs.

In [ ]:
#export
def identify_available_datasets(start_date: str, end_date: str, 
                                product_id='EO:EUM:DAT:MSG:MSG15-RSS'):
    """
    Identifies available datasets from the EUMETSAT data
    API for the specified data product and date-range. 
    The dates will accept any format that can be 
    interpreted by `pd.to_datetime`.
    
    Parameters:
        start_date: Start of the query period
        end_date: End of the query period
        product_id: ID of the EUMETSAT product requested
        
    Returns:
        r: Response from the request
        
    """
    
    num_features = 10000
    start_index = 0
    
    datasets = []
    all_results_returned = False
    
    while all_results_returned == False:
        r_json = query_data_products(start_date, end_date, 
                                     start_index=start_index, 
                                     num_features=num_features, 
                                     product_id='EO:EUM:DAT:MSG:MSG15-RSS').json()

        datasets += r_json['features']

        num_total_results = r_json['properties']['totalResults']
        num_returned_results = start_index + len(r_json['features'])

        if num_returned_results < num_total_results:
            start_index += num_features
        else:
            all_results_returned = True
            
        assert num_returned_results == len(datasets), 'Some features have not been appended'
        
    return datasets

<br>

We'll check that the same number of available datasets are identified

In [ ]:
%%time

datasets = identify_available_datasets(start_date, end_date)

print(f'{len(datasets)} datasets have been identified')

<br>

Finally we'll create a helper function for converting the dataset ids into their file urls.

In [ ]:
#export
dataset_id_to_link = lambda data_id: f'https://api.eumetsat.int/data/download/products/{data_id}'

<br>

We'll now test this works.

N.b. You cannot use the link returned here directly as it will not be OAuth'ed

In [ ]:
dataset_ids = sorted([dataset['id'] for dataset in datasets])
example_data_link = dataset_id_to_link(dataset_ids[0])

example_data_link

<br>

### Downloading Data

Now that we know where our data is located we want to download it. First we'll check that the directory we wish to save the data in exists, if not we'll create it

In [ ]:
for folder in [data_dir, sorted_dir]:
    if not os.path.exists(folder):
        os.makedirs(folder)

<br>

We also want to extract the relevant metadata information from each file. Here we'll create a generalised framework for extracting data from any product, to add a new one please add its metadata mapping under the relevant `product_id`.

In [ ]:
# export
def json_extract(json_obj:Union[dict, list], locators:list):
    extracted_obj = copy.deepcopy(json_obj)
    
    for locator in locators:
        extracted_obj = extracted_obj[locator]
        
    return extracted_obj

def extract_metadata(data_dir: str, product_id='EO:EUM:DAT:MSG:MSG15-RSS'):
    with open(f'{data_dir}/EOPMetadata.xml', 'r') as f:
        xml_str = f.read()
        
    raw_metadata = xmltodict.parse(xml_str)
    metadata_map = metadata_maps[product_id]
    
    datatypes_to_transform_func = {
        'datetime': pd.to_datetime,
        'str': str,
        'int': int,
        'float': float
    }
    
    cleaned_metadata = dict()

    for feature, attrs in metadata_map.items():
        location = attrs['location']
        datatype = attrs['datatype']

        value = json_extract(raw_metadata, location)
        formatted_value = datatypes_to_transform_func[datatype](value)

        cleaned_metadata[feature] = formatted_value

    return cleaned_metadata

metadata_maps = {
    'EO:EUM:DAT:MSG:MSG15-RSS': {
        'start_date': {
            'datatype': 'datetime',
            'location': ['eum:EarthObservation', 'om:phenomenonTime', 'gml:TimePeriod', 'gml:beginPosition']
        },
        'end_date': {
            'datatype': 'datetime',
            'location': ['eum:EarthObservation', 'om:phenomenonTime', 'gml:TimePeriod', 'gml:endPosition']
        },
        'result_time': {
            'datatype': 'datetime',
            'location': ['eum:EarthObservation', 'om:resultTime', 'gml:TimeInstant', 'gml:timePosition']
        },
        'platform_short_name': {
            'datatype': 'str',
            'location': ['eum:EarthObservation', 'om:procedure', 'eop:EarthObservationEquipment', 'eop:platform', 'eop:Platform', 'eop:shortName']
        },
        'platform_orbit_type': {
            'datatype': 'str',
            'location': ['eum:EarthObservation', 'om:procedure', 'eop:EarthObservationEquipment', 'eop:platform', 'eop:Platform', 'eop:orbitType']
        },
        'instrument_name': {
            'datatype': 'str',
            'location': ['eum:EarthObservation', 'om:procedure', 'eop:EarthObservationEquipment', 'eop:instrument', 'eop:Instrument', 'eop:shortName']
        },
        'sensor_op_mode': {
            'datatype': 'str',
            'location': ['eum:EarthObservation', 'om:procedure', 'eop:EarthObservationEquipment', 'eop:sensor', 'eop:Sensor', 'eop:operationalMode']
        },
        'center_srs_name': {
            'datatype': 'str',
            'location': ['eum:EarthObservation', 'om:featureOfInterest', 'eop:Footprint', 'eop:centerOf', 'gml:Point', '@srsName']
        },
        'center_position': {
            'datatype': 'str',
            'location': ['eum:EarthObservation', 'om:featureOfInterest', 'eop:Footprint', 'eop:centerOf', 'gml:Point', 'gml:pos']
        },
        'file_name': {
            'datatype': 'str',
            'location': ['eum:EarthObservation', 'om:result', 'eop:EarthObservationResult', 'eop:product', 'eop:ProductInformation', 'eop:fileName', 'ows:ServiceReference', '@xlink:href']
        },
        'file_size': {
            'datatype': 'int',
            'location': ['eum:EarthObservation', 'om:result', 'eop:EarthObservationResult', 'eop:product', 'eop:ProductInformation', 'eop:size', '#text']
        },
        'missing_pct': {
            'datatype': 'float',
            'location': ['eum:EarthObservation', 'eop:metaDataProperty', 'eum:EarthObservationMetaData', 'eum:missingData', '#text']
        },
    }
}

<br>

We're now ready to create a download manager that will handle all of the querying, processing and retrieving for us

In [ ]:
#export
def check_valid_request(r: requests.models.Response):
    """
    Checks that the response from the request is valid
    
    Parameters:
        r: Response object from the request

    """
    
    class InvalidCredentials(Exception):
        pass

    if r.ok == False:
        if 'Invalid Credentials' in r.text:
            raise InvalidCredentials('The access token passed in the API request is invalid')
        else:
            raise Exception(f'The API request was unsuccesful {r.text} {r.status_code}')
            
    return

class DownloadManager:
    """
    The DownloadManager class provides a handler for downloading data
    from the EUMETSAT API, managing: retrieval, logging and metadata
    
    """
    
    def __init__(self, user_key: str, user_secret: str, 
                 data_dir: str, metadata_db_fp: str, log_fp: str, 
                 main_logging_level: str='DEBUG', slack_logging_level: str='CRITICAL', 
                 slack_webhook_url: str=None, slack_id: str=None, 
                 bucket_name=None, bucket_prefix=None, logger_name='EUMETSAT Download'):
        """
        Initialises the download manager by:
        * Setting up the logger
        * Requesting an API access token
        * Configuring the download directory
        * Connecting to the metadata database
        * Adding satip helper functions

        Parameters:
            user_key: EUMETSAT API key
            user_secret: EUMETSAT API secret
            data_dir: Path to the directory where the satellite data will be saved
            metadata_db_fp: Path to where the metadata database is stored/will be saved
            log_fp: Filepath where the logs will be stored
            main_logging_level: Logging level for file and Jupyter
            slack_logging_level: Logging level for Slack
            slack_webhook_url: Webhook for the log Slack channel
            slack_id: Option user-id to mention in Slack
            bucket_name: (Optional) Google Cloud Storage bucket name to check for existing files
            bucket_prefix: (Optional) Prefix for cloud bucket files

        Returns:
            download_manager: Instance of the DownloadManager class

        """
        
        # Configuring the logger
        self.logger = utils.set_up_logging(logger_name, log_fp, 
                                           main_logging_level, slack_logging_level, 
                                           slack_webhook_url, slack_id)
        
        self.logger.info(f'********** Download Manager Initialised **************')
        
        # Requesting the API access token
        self.user_key = user_key
        self.user_secret = user_secret
        
        self.request_access_token()
        
        # Configuring the data directory
        self.data_dir = data_dir
        
        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)
        
        # Initialising the metadata database
        self.metadata_db = dataset.connect(f'sqlite:///{metadata_db_fp}')
        self.metadata_table = self.metadata_db['metadata']
        
        # Adding satip helper functions
        self.identify_available_datasets = identify_available_datasets
        self.query_data_products = query_data_products

        # Google Cloud integration
        self.bucket_name = bucket_name
        self.bucket_prefix = bucket_prefix
        self.bucket_filenames = None

        if bucket_name:
            print(f'Checking files in GCP bucket {bucket_name}, this will take a few seconds')
            filenames = get_eumetsat_filenames(bucket_name, prefix=bucket_prefix)
            self.bucket_filenames = [re.match("([A-Z\d.]+-){6}", filename)[0][:-1] for filename in filenames]
        
        return
    
    
    def request_access_token(self, user_key=None, user_secret=None): 
        """
        Requests an access token from the EUMETSAT data API.
        If no key or secret are provided then they will default
        to the values provided in the download manager initialisation

        Parameters:
            user_key: EUMETSAT API key
            user_secret: EUMETSAT API secret

        Returns:
            access_token: API access token

        """
    
        if user_key is None:
            user_key = self.user_key
        if user_secret is None:
            user_secret = self.user_secret
            
        self.access_token = request_access_token(user_key, user_secret)
        
        return
        
    def download_single_dataset(self, data_link:str):
        """
        Downloads a single dataset from the EUMETSAT API

        Parameters:
            data_link: Url link for the relevant dataset
        
        """
        
        params = {
            'access_token': self.access_token
        }

        r = requests.get(data_link, params=params)
        check_valid_request(r)

        zipped_files = zipfile.ZipFile(BytesIO(r.content))
        zipped_files.extractall(f'{self.data_dir}')

        return


    def check_if_downloaded(self, filenames: List[str]):
        """Checks which files should be downloaded based on 
        local file contents and a cloud storage bucket, if specified.
        
        Parameters:
            filenames: List of filename strings
        
        Returns:
            List of filenames to download
        """
        in_bucket = []
        local = []
        download = []
                
        for filename in filenames:
            # get first part of filename for matching
            match = re.match("([A-Z\d.]+-){6}", filename)[0][:-1]
            
            if self.bucket_name:
                if match in self.bucket_filenames:
                    in_bucket.append(filename)
                    if f'{filename}.nat' in os.listdir(self.data_dir):
                        local.append(filename)
                    continue

            if f'{filename}.nat' in os.listdir(self.data_dir):
                local.append(filename)
                continue
                
            download.append(filename)

        if self.bucket_name:
            self.logger.info(f'{len(filenames)} files queried, {len(in_bucket)} found in bucket, {len(local)} found in {self.data_dir}, {len(download)} to download.')
        else:
            self.logger.info(f'{len(filenames)} files queried, {len(local)} found in {self.data_dir}, {len(download)} to download.')

        return download, local


    def download_date_range(self, start_date:str, end_date:str, product_id='EO:EUM:DAT:MSG:MSG15-RSS'):
        """
        Downloads a set of dataset from the EUMETSAT API
        in the defined date range and specified product

        Parameters:
            start_date: Start of the requested data period
            end_date: End of the requested data period
            product_id: ID of the EUMETSAT product requested
        
        """
    
        datasets = identify_available_datasets(start_date, end_date, product_id=product_id)
        df_new_metadata = self.download_datasets(datasets, product_id=product_id)

        return df_new_metadata        


    def download_datasets(self, datasets, product_id='EO:EUM:DAT:MSG:MSG15-RSS', download_all=True):
        """
        Downloads a set of dataset from the EUMETSAT API
        in the defined date range and specified product

        Parameters:
            datasets: list of datasets returned by `identify_available_datasets`
        
        """

        # Identifying dataset ids to download
        dataset_ids = sorted([dataset['id'] for dataset in datasets])

        # Check which datasets to download
        download_ids, local_ids = self.check_if_downloaded(dataset_ids)
        
        # Downloading specified datasets
        if not dataset_ids:
            self.logger.info('No files will be downloaded. Set DownloadManager bucket_name argument for local download')
            return 
        
        all_metadata = []

        for dataset_id in track(dataset_ids):
            dataset_link = dataset_id_to_link(dataset_id)

            # Download the raw data
            if (dataset_id in download_ids) or (download_all == True):
                try:
                    self.download_single_dataset(dataset_link)
                except:
                    self.logger.info('The EUMETSAT access token has been refreshed')
                    self.request_access_token()
                    self.download_single_dataset(dataset_link)

            # Extract and save metadata
            dataset_metadata = extract_metadata(self.data_dir, product_id=product_id)
            dataset_metadata.update({'downloaded': pd.Timestamp.now()})
            all_metadata += [dataset_metadata]
            self.metadata_table.insert(dataset_metadata)

            # Delete old metadata files
            for xml_file in ['EOPMetadata.xml', 'manifest.xml']:
                xml_filepath = f'{self.data_dir}/{xml_file}'

                if os.path.isfile(xml_filepath):
                    os.remove(xml_filepath)
                    
        df_new_metadata = pd.DataFrame(all_metadata)
                    
        return df_new_metadata
    
    get_df_metadata = lambda self: pd.DataFrame(self.metadata_table.all()).set_index('id')

<br>

We'll now see what it looks like when we initialise the download manager

In [ ]:
dm = DownloadManager(user_key, user_secret, data_dir, metadata_db_fp, debug_fp, 
                     slack_webhook_url=slack_webhook_url, slack_id=slack_id)

start_date = '2020-10-01 12:00'
end_date = '2020-10-01 12:05'

if download_data == True:
    dm.download_date_range(start_date, end_date)

df_metadata = dm.get_df_metadata()

df_metadata.head()

<br>

The `get_size` function was adapted from <a href="https://stackoverflow.com/questions/1392413/calculating-a-directorys-size-using-python">this stackoverflow answer</a>

In [ ]:
#export
def get_dir_size(directory='.'):
    total_size = 0
    
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

In [ ]:
data_dir_size = get_dir_size(data_dir)

print(f'The data directory is currently {round(data_dir_size/1_000_000_000, 2):,} Gb')

<br>
<br>
<br>
<br>

# Need to Clean Notebook Below Here

<br>
<br>
<br>
<br>

## Integration with Google Cloud Storage

If Google Cloud Platform (GCP) flags are passed (`bucket_name` and `bucket_prefix`), when downloading, then the `DownloadManager` should first check to see if the files already exist in the specified cloud storage bucket.  
If the files already exist, then they will not be downloaded locally - if the storage bucket arguments are passed.  

In [ ]:
BUCKET_NAME = "solar-pv-nowcasting-data"
PREFIX = "satellite/EUMETSAT/SEVIRI_RSS/native/2020"

In [ ]:
dm = DownloadManager(user_key, user_secret, data_dir, metadata_db_fp, debug_fp, bucket_name=BUCKET_NAME, bucket_prefix=PREFIX)

In [ ]:
# Bucket filenames can be accessed
len(dm.bucket_filenames)

Lets test this by examining some dates at the start of 2020

In [ ]:
# Timings: around 2 hours to download 1 day.

# DownloadManager should find these 2020 files on the VM
start_date = '2020-01-01 00:00'
end_date = '2020-01-02 00:00'
dm.download_date_range(start_date, end_date)

## Name Convention changes in EUMETSAT files

Probably due to the changes / creation of the EUMETSAT API around the end of 2019, newer files do not contain the previous 'order number' parts at the end of the filename. 

Some previously downloaded files follow a different file name convention:

Files through new API:
`MSG3-SEVI-MSG15-0100-NA-20191001120415.883000000Z-NA.nat`  
SatProgram-Instrument-SatNumber-AlgoVersion-InstrumentMode(?)-ReceptionStartDateUTC   

Files on GCP:  
` MSG3-SEVI-MSG15-0100-NA-20191001120415.883000000Z-20191001120433-1399526-1.nat.bz2`  
SatProgram-Instrument-SatNumber-AlgoVersion-InstrumentMode(?)-ReceptionStartDateUTC-SensingStartDateUTC-OrderNumber-PartNumber 

We can use regex to take the first part of the filename for comparisons

In [ ]:
txt = "MSG3-SEVI-MSG15-0100-NA-20190101000417.314000000Z-20190101000435-1377854-1.nat"
re.match("([A-Z\d.]+-){6}", txt)[0][:-1] # [:-1] to trim the trailing -

To ensure we are comparing the same filenames, this regex is added into DownloadManager and the GCP helpers.

Set up logging locally.

In [ ]:
debug_fp = '../logs/EUMETSAT_download.txt'
log = utils.set_up_logging('EUMETSAT Processing', debug_fp)

## Helper utils

In [ ]:
#export 
def get_filesize_megabytes(filename):
    filesize_bytes = os.path.getsize(filename)
    return filesize_bytes / 1E6


def eumetsat_filename_to_datetime(inner_tar_name):
    p = re.compile('^MSG[23]-SEVI-MSG15-0100-NA-(\d*)\.')
    title_match = p.match(inner_tar_name)
    date_str = title_match.group(1)
    return datetime.datetime.strptime(date_str, "%Y%m%d%H%M%S")

For local testing, this command downloads some files from the Google Cloud bucket:

In [ ]:
# get test files from GCP - these are compressed
# !gsutil cp -r gs://solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/native/2019/10/01/00/04 ../data/raw

## Compress downloaded native image files

Once files have been downloaded from the EUMETSAT API in some location, they need to be compressed and saved in a cloud storage bucket. 

First, see which files have already been downloaded locally to test this functionality.  

In [ ]:
full_native_filenames = glob.glob(os.path.join(data_dir, '*.nat'))
full_native_filenames

We will compress locally downloaded files here using `pbzip2`  
On ubuntu: `sudo apt-get install -y pbzip2`  
On mac: `brew install pbzip2`  

In [ ]:
#export
def compress_downloaded_files(data_dir, compressed_dir, log=None):
    """
    Compresses downloaded files, stores them locally, 
    and ensures they are approximately the correct filesize.
    Uses pbzip2 for compression.

        Parameters:
            data_dir: (string), directory path containing raw downloaded files from EUMETSAT API
            compressed_dir: (string), directory path for compressed .nat files 
            log: (bool), flag to enable logging

        Returns:
            -
    """
    NATIVE_FILESIZE_MB = 102.210123
    EXTENSION = '.bz2'
    
    full_native_filenames = glob.glob(os.path.join(data_dir, '*.nat'))
    print(f'Found {len(full_native_filenames)} native files.')
    if log:
        log.info(f'Found {len(full_native_filenames)} native files.')

    for full_native_filename in full_native_filenames:
        # Check filesize is correct
        native_filesize_mb = get_filesize_megabytes(full_native_filename)

        if not math.isclose(native_filesize_mb, NATIVE_FILESIZE_MB, abs_tol=1):
            msg = f'Filesize incorrect for {full_native_filename}!  Expected {NATIVE_FILESIZE_MB} MB.  Actual = {native_filesize_mb} MB.'
            if log:
                log.error(msg)

        if log:
            log.debug('Compressing %s', full_native_filename)

        completed_process = subprocess.run(['pbzip2', '-5', full_native_filename])
        try:
            completed_process.check_returncode()
        except:
            if log:
                log.exception('Compression failed!')
            print('Compression failed!')
            raise

        full_compressed_filename = full_native_filename + EXTENSION
        compressed_filesize_mb = get_filesize_megabytes(full_compressed_filename)
        if log:
            log.debug(f'Filesizes: Before compression = {native_filesize_mb} MB. After compression = {compressed_filesize_mb} MB.  Compressed file is {compressed_filesize_mb/native_filesize_mb} x the size of the uncompressed file.')

        base_native_filename = os.path.basename(full_native_filename)
        dt = eumetsat_filename_to_datetime(base_native_filename)
        
        # Creating compressed_dir if not already made
        if not os.path.exists(compressed_dir):
            os.makedirs(compressed_dir)
    
        new_dst_path = os.path.join(compressed_dir, dt.strftime("%Y/%m/%d/%H/%M"))
        if not os.path.exists(new_dst_path):
            os.makedirs(new_dst_path)

        new_dst_full_filename = os.path.join(new_dst_path, base_native_filename + EXTENSION)
        if log:
            log.debug('Moving %s to %s', full_compressed_filename, new_dst_full_filename)
        
        if os.path.exists(new_dst_full_filename):
            if log:
                log.debug(f'{new_dst_full_filename} already exists.  Deleting old file')
            os.remove(new_dst_full_filename)
        shutil.move(src=full_compressed_filename, dst=new_dst_path)

In [ ]:
compress_downloaded_files(data_dir=data_dir, compressed_dir=compressed_dir)

## Syncing files to GCP Storage

Compressed native files should be stored in a Google Cloud Storage Bucket. The folder structure follows the following convention:  

`gs://solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/native/<year>/<month>/<day>/<hour>/<minute>/`

In [ ]:
# sync downloaded files in compressed_dir to the bucket
BUCKET_NAME = "solar-pv-nowcasting-data"
PREFIX = "satellite/EUMETSAT/SEVIRI_RSS/native/"

In [ ]:
# export
def upload_compressed_files(compressed_dir, BUCKET_NAME, PREFIX, log=None):
    """Uploads compressed native files to a Google Cloud storage bucket

    For example,
    compressed_dir:  /home/srv/data/intermediate/
    bucket name: solar-pv-nowcasting-data
    prefix:      satellite/EUMETSAT/SEVIRI_RSS/native/

    With some files like:
    /home/srv/data/intermediate/2018/01/01/01/23/04/MSG3-SEVI-MSG15-0100-NA-20191001120415.883000000Z-NA.nat.bz2

    Would upload the files to:
    gs://solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/native/2018/01/01/01/23/04/MSG3-SEVI-MSG15-0100-NA-20191001120415.883000000Z-NA.nat.bz2
    etc

        Parameters:
            compressed_dir: (str), directory where compressed files are stored locally
            BUCKET_NAME: (str), name of Google Cloud storage bucket
            PREFIX: (str), string prefix to use as part of the bucket storage path

        Returns:
            -
    """
    paths = Path(compressed_dir).rglob("*.nat.bz2")
    full_compressed_files = [x for x in paths if x.is_file()]
    if log:
        log.info(f"Found {len(full_compressed_files)} compressed files.")

    for file in full_compressed_files:
        rel_path = os.path.relpath(file.absolute(), compressed_dir)
        upload_blob(
            bucket_name=BUCKET_NAME,
            source_file_name=file.absolute(),
            destination_blob_name=rel_path,
            prefix=PREFIX,
        )

In [ ]:
#hide
from nbdev.export import *
notebook2script()